In [1]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import *

import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime
import re
import os
import glob
import time
import pickle

In [2]:
homepage = "https://www.airbnb.ca"

In [3]:
city = 'Toronto'

In [4]:
check_in_date = '2021-04-11'
check_out_date = '2021-04-15'

In [5]:
no_adult = 4
no_kid = 1

In [6]:
def load_page(page):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get(page)
    return driver

In [7]:
def search_places(city, check_in_date, check_out_date, no_adult, no_kid):
    driver = load_page(homepage)
    p = driver.find_element_by_id("bigsearch-query-detached-query")
    p.send_keys(city, Keys.ENTER)
    
    driver.implicitly_wait(5)
    
    ci_date = datetime.strptime(check_in_date, '%Y-%m-%d').strftime('%B %d, %Y')
    co_date = datetime.strptime(check_out_date, '%Y-%m-%d').strftime('%B %d, %Y')
    ci = driver.find_element_by_xpath("//td[@class='_12fun97'][contains(@aria-label, '{}')]".format(ci_date)).send_keys(Keys.RETURN)
    co = driver.find_element_by_xpath("//td[@class='_12fun97'][contains(@aria-label, '{}')]".format(co_date)).send_keys(Keys.RETURN)
    
    driver.find_element_by_xpath("//div[@data-testid='structured-search-input-field-guests-button']").click()
    
    for n in range(no_adult):    
        driver.find_element_by_xpath("//button[@class='_7hhhl3'][@data-testid='stepper-adults-increase-button']").click()
    for n in range(no_kid):
        driver.find_element_by_xpath("//button[@class='_7hhhl3'][@data-testid='stepper-children-increase-button']").click()
        
    window_old = driver.window_handles[0]
    result = driver.find_element_by_xpath("//button[@class='_1mzhry13'][@data-testid='structured-search-input-search-button']").click()
    window_new = driver.window_handles[0]
    driver.switch_to.window(window_new)
    
    driver.implicitly_wait(5)
    
    listing = []
    no_of_page = driver.find_element_by_xpath("//div[@aria-label='Search results pagination']").find_elements_by_tag_name('a')[-2].text
    for p in range(int(no_of_page) - 1):
        link_list = get_list_link(driver, homepage)
        for l in link_list:
            listing.append(l)
        driver.find_element_by_xpath("//a[@aria-label='Next']").send_keys(Keys.RETURN)
        driver.implicitly_wait(5)
#         win_new = driver.window_handles[0]
#         driver.switch_to.window(win_new)
        current_url = driver.current_url
        driver.get(current_url)
        
    return listing

In [8]:
def gen_soup(url):
        page = requests.get(url, headers={"User-Agent": "XY"}).text
        soup = BeautifulSoup(page, 'html.parser')
        return soup

In [9]:
def get_list_link(page, homepage):
    soup = BeautifulSoup(page.page_source, 'html.parser')
    link_list = [homepage + t.a.get('href') for t in soup.find_all('div', class_='_8s3ctt')]
    return link_list

In [80]:
def get_reviews(url):
    page = load_page(url)
    page.implicitly_wait(5)
    no_reviews = re.findall('\d+', page.find_element_by_class_name('_owhb5o').text)[0]
    page.find_element_by_link_text('Show all {} reviews'.format(no_reviews)).send_keys(Keys.RETURN)
    page.implicitly_wait(3)
    current_url = page.current_url
    page.get(current_url)
    
    reviews = []
    
    loaded_reviews = page.find_elements_by_xpath("//div[@class='_1d784e5']/span")
    loaded_till_now = len(loaded_reviews)

    while(loaded_till_now < no_reviews):
        loaded_reviews[loaded_till_now-1].location_once_scrolled_into_view
        page.find_element_by_tag_name('body').send_keys(Keys.SPACE)
        page.implicitly_wait(5)
        loaded_reviews = page.find_elements_by_xpath("//div[@class='_1d784e5']/span")
        loaded_till_now = len(loaded_reviews)
        
    for lr in loaded_reviews:
        reviews.append(lr.text)
        
    return reviews

In [11]:
listing = search_places(city, check_in_date, check_out_date, no_adult, no_kid)

In [82]:
url = listing[2]

In [86]:
page002 = load_page(url)

In [83]:
reviews = get_reviews(url)